

### 🏷️ **Credits & License**

* 🔗 [Chatterbox GitHub Repository](https://github.com/resemble-ai/chatterbox)
* 🤗 [Chatterbox on Hugging Face](https://huggingface.co/ResembleAI/chatterbox)
* 📄 **License**: Provided under the [MIT License](https://github.com/resemble-ai/chatterbox/blob/master/LICENSE)




### ⚠️ **Usage Disclaimer**

Use of this voice cloning model is subject to strict ethical and legal standards. By using this tool, you agree **not to** engage in any of the following prohibited activities:

* **Fraud or Deception**: Using cloned voices to create misleading or fraudulent content.
* **Impersonation**: Replicating someone’s voice without their explicit permission, especially for malicious, harmful, or deceptive purposes.
* **Illegal Activities**: Employing the model in any manner that violates local, national, or international laws and regulations.
* **Harmful Content Generation**: Creating offensive, defamatory, or unethical material, including content that spreads misinformation or causes harm.

> ⚖️ **Legal Responsibility**
> The developers of this tool disclaim all liability for misuse. **Users bear full responsibility** for ensuring that their usage complies with all applicable laws, regulations, and ethical guidelines.




In [ ]:
#@title Install ChatterBox
%cd /content/
# !git clone https://github.com/resemble-ai/chatterbox.git
!git clone https://github.com/NeuralFalconYT/chatterbox.git


def disable_watermark(file_path="./chatterbox/src/chatterbox/tts.py", disable=True):
    if not disable:
        print("Watermarking remains enabled. No changes made.")
        return

    with open(file_path, "r") as f:
        code = f.read()

    target_line = "watermarked_wav = self.watermarker.apply_watermark(wav, sample_rate=self.sr)"
    replacement_line = "watermarked_wav = wav"

    if target_line in code:
        code = code.replace(target_line, replacement_line)
        with open(file_path, "w") as f:
            f.write(code)
        print("✅ Watermarking disabled successfully.")
    else:
        print("ℹ️ Watermarking already disabled or line not found.")

def fix_chrome_hang(file_path = "./chatterbox/src/chatterbox/models/t3/t3.py"):
    try:
        with open(file_path, 'r') as f:
            content = f.read()

        content = content.replace(
            'for i in tqdm(range(max_new_tokens), desc="Sampling", dynamic_ncols=True):',
            'for i in range(max_new_tokens):'
        )

        with open(file_path, 'w') as f:
            f.write(content)

        print("✅ tqdm replaced with a normal for loop to fix Chrome hang.")
    except Exception as e:
        print(f"❌ Failed to patch file: {e}")
fix_chrome_hang()

## If you use your own voice recording and do NOT want the AI watermark,
## uncomment the following line to disable it:

# disable_watermark(disable=True)


%cd ./chatterbox
!wget https://raw.githubusercontent.com/NeuralFalconYT/chatterbox-colab/refs/heads/main/app.py

!mkdir /content/uploaded_audio
%cd /content/uploaded_audio
!wget https://storage.googleapis.com/chatterbox-demo-samples/prompts/female_shadowheart4.flac


!pip install numpy~=1.26.0
!pip install resampy==0.4.3
!pip install librosa==0.11.0
!pip install s3tokenizer
# !pip install torch==2.6.0
# !pip install torchaudio==2.6.0
!pip install transformers==4.46.3
!pip install diffusers==0.29.0
!pip install resemble-perth==1.0.1
!pip install omegaconf==2.3.0
!pip install conformer==0.3.2
!pip install safetensors==0.5.3
!pip install sentence-splitter
!pip install gradio>=5.9.1
!pip install soundfile

from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

Don't panic if you see this image.png red error cell, that just means Colab automatically restarted the kernel. Now, simply run the next cell.

In [ ]:
#@title Gradio App
%cd /content/chatterbox
!python app.py --share #--debug

#Run From Colab Cell

In [3]:
#@title Mount Google Drive for auto save [ Optional ]
%cd /content/
from IPython.display import clear_output
from google.colab import drive
import os

auto_save_in_drive = False  # @param {type: "boolean"}

if auto_save_in_drive:
    drive.mount('/content/gdrive', force_remount=True)
    os.makedirs("/content/gdrive/MyDrive/colab_download", exist_ok=True)
    clear_output()
    print("✅ A copy of the cloned voice will be automatically saved to your Google Drive.")



/content


In [4]:
#@title Download Model
%cd /content/chatterbox
# my useless funtions
import random
import numpy as np
import torch
import gradio as gr
import gc
import subprocess
# Initialize default pipeline
temp_audio_dir="./cloned_voices"
os.makedirs(temp_audio_dir, exist_ok=True)
def get_max_gpu_memory():
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory
        return round(total_memory / (1024 ** 3), 2)  # Convert to GB
    else:
        print("CUDA is not available.")
        return None

def is_gpu_memory_over_limit(safety_margin_gb=0.8):
    """
    Returns True if GPU memory usage exceeds (total - safety_margin).
    """
    max_memory_gb = get_max_gpu_memory()
    if max_memory_gb is None:
        return False  # Can't check memory if no GPU

    limit_gb = max_memory_gb - safety_margin_gb

    try:
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        memory_used_mb_list = result.stdout.strip().splitlines()
        for i, memory_used_mb in enumerate(memory_used_mb_list):
            memory_used_gb = int(memory_used_mb) / 1024.0
            if memory_used_gb > limit_gb:
                print(f"Maximum GPU memory available: {max_memory_gb} GB")
                print(f"Memory limit set: {limit_gb} GB")
                print("⚠️ GPU memory usage has exceeded the safe threshold. Clearing memory and reloading the model...")
                return True
        # print("✅ GPU memory is within safe limits.")
        return False
    except Exception as e:
        print(f"Failed to check GPU memory: {e}")
        return False



from sentence_splitter import SentenceSplitter
import re
import uuid
import numpy as np
import wave
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os


def word_split(text, char_limit=300):
    words = text.split()
    chunks = []
    current_chunk = ""

    for word in words:
        if len(current_chunk) + len(word) + (1 if current_chunk else 0) <= char_limit:
            current_chunk += (" " if current_chunk else "") + word
        else:
            chunks.append(current_chunk)
            current_chunk = word

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

def split_into_chunks(text, max_char_limit=300):
    if len(text)>=300:
      print("⚠️ The text is too long. Breaking it into smaller pieces so the voice generation works correctly.")

      splitter = SentenceSplitter(language='en')
      raw_sentences = splitter.split(text)

      # Flattened list of sentence-level word chunks
      sentence_chunks = []
      for sen in raw_sentences:
          sentence_chunks.extend(word_split(sen, char_limit=max_char_limit))

      chunks = []
      temp_str = ""

      for sentence in sentence_chunks:
          if len(temp_str) + len(sentence) + (1 if temp_str else 0) <= max_char_limit:
              temp_str += (" " if temp_str else "") + sentence
          else:
              chunks.append(temp_str)
              temp_str = sentence

      if temp_str:
          chunks.append(temp_str)

      return chunks
    else:
      return [text]


def clean_text(text):
    # Define replacement rules
    replacements = {
        "–": " ",  # Replace en-dash with space
        "—": " ",  #
        "-": " ",  # Replace hyphen with space
        "**": " ", # Replace double asterisks with space
        "*": " ",  # Replace single asterisk with space
        "#": " ",  # Replace hash with space
    }

    # Apply replacements
    for old, new in replacements.items():
        text = text.replace(old, new)

    # Remove emojis using regex (covering wide range of Unicode characters)
    emoji_pattern = re.compile(
        r'[\U0001F600-\U0001F64F]|'  # Emoticons
        r'[\U0001F300-\U0001F5FF]|'  # Miscellaneous symbols and pictographs
        r'[\U0001F680-\U0001F6FF]|'  # Transport and map symbols
        r'[\U0001F700-\U0001F77F]|'  # Alchemical symbols
        r'[\U0001F780-\U0001F7FF]|'  # Geometric shapes extended
        r'[\U0001F800-\U0001F8FF]|'  # Supplemental arrows-C
        r'[\U0001F900-\U0001F9FF]|'  # Supplemental symbols and pictographs
        r'[\U0001FA00-\U0001FA6F]|'  # Chess symbols
        r'[\U0001FA70-\U0001FAFF]|'  # Symbols and pictographs extended-A
        r'[\U00002702-\U000027B0]|'  # Dingbats
        r'[\U0001F1E0-\U0001F1FF]'   # Flags (iOS)
        r'', flags=re.UNICODE)

    text = emoji_pattern.sub(r'', text)

    # Remove multiple spaces and extra line breaks
    text = re.sub(r'\s+', ' ', text).strip()

    return text




def tts_file_name(text, language="en"):
    global temp_audio_dir
    # Clean and process the text
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only letters and spaces
    text = text.lower().strip().replace(" ", "_")

    # Ensure the text is not empty
    if not text:
        text = "audio"

    # Truncate to first 20 characters for filename
    truncated_text = text[:20]

    # Sanitize and format the language tag
    language = re.sub(r'\s+', '_', language.strip().lower()) if language else "unknown"

    # Generate random suffix
    random_string = uuid.uuid4().hex[:8].upper()

    # Construct the filename
    file_name = f"{temp_audio_dir}/{truncated_text}_{language}_{random_string}.wav"
    return file_name








def remove_silence_function(file_path,minimum_silence=50):
    # Extract file name and format from the provided path
    output_path = file_path.replace(".wav", "_no_silence.wav")
    audio_format = "wav"
    # Reading and splitting the audio file into chunks
    sound = AudioSegment.from_file(file_path, format=audio_format)
    audio_chunks = split_on_silence(sound,
                                    min_silence_len=100,
                                    silence_thresh=-45,
                                    keep_silence=minimum_silence)
    # Putting the file back together
    combined = AudioSegment.empty()
    for chunk in audio_chunks:
        combined += chunk
    combined.export(output_path, format=audio_format)
    return output_path

#chatterbox code
from src.chatterbox.tts import ChatterboxTTS


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

chatterbox_model=None
def load_model():
    global chatterbox_model
    if chatterbox_model is None:
      del chatterbox_model
      gc.collect()
      torch.cuda.empty_cache()
    chatterbox_model = ChatterboxTTS.from_pretrained(DEVICE)
    return chatterbox_model

# for the first time
chatterbox_model=load_model()

import tempfile
import shutil
import os
import numpy as np
import soundfile as sf
from tqdm.auto import tqdm

def generate(chatterbox_model, text, audio_prompt_path, exaggeration, temperature, seed_num, cfgw):
    if is_gpu_memory_over_limit():
        chatterbox_model = load_model()

    # Removed seed setting here to avoid conflicts
    # if seed_num != 0:
    #         set_seed(int(seed_num))

    wav = chatterbox_model.generate(
        text,
        audio_prompt_path=audio_prompt_path,
        exaggeration=exaggeration,
        temperature=temperature,
        cfg_weight=cfgw,
    )
    sr = chatterbox_model.sr
    audio = wav.squeeze(0).numpy()
    return sr, audio

def generate_and_save_all(text, audio_prompt_path, exaggeration, temperature, seed_num, cfgw):
    global chatterbox_model
    text = clean_text(text)
    chunks = split_into_chunks(text, max_char_limit=300)

    temp_dir = tempfile.mkdtemp(prefix="audio_chunks_")
    temp_files = []

    try:
        for idx, chunk in tqdm(enumerate(chunks), total=len(chunks), desc="Generating audio"):
            try:
                # Set seed once per chunk for reproducible variation
                if seed_num != 0:
                    set_seed(int(seed_num) + idx)

                sr, audio = generate(
                    chatterbox_model,
                    chunk,
                    audio_prompt_path,
                    exaggeration,
                    temperature,
                    seed_num=0,  # Pass 0 to skip seed setting inside generate
                    cfgw=cfgw
                )

                chunk_path = os.path.join(temp_dir, f"chunk_{idx:03}.wav")
                sf.write(chunk_path, audio, sr)
                temp_files.append(chunk_path)

            except Exception as e:
                print(f"⚠️ [Chunk {idx}] Generation failed: {e}")
                print(f"Text: {chunk}")
                print(f"Length: {len(chunk)}")
                continue  # Skip failed chunk

        # Merge all valid chunks
        final_audio = []
        for file_path in temp_files:
            try:
                data, _ = sf.read(file_path)
                final_audio.append(data)
            except Exception as e:
                print(f"💀 [Merging] Failed to read chunk: {file_path} ({e})")

        if final_audio:
            final_audio = np.concatenate(final_audio)
            final_path = tts_file_name(text, language="en")
            sf.write(final_path, final_audio, sr)
        else:
            raise RuntimeError("All audio chunk generations failed.")

    finally:
        shutil.rmtree(temp_dir)

    return final_path


from IPython.display import Audio, display

def play_audio(audio_path, autoplay=False):
    display(Audio(audio_path, autoplay=autoplay))



from IPython.display import clear_output
clear_output()
print("✅ The model has been downloaded and initialized successfully.")

✅ The model has been downloaded and initialized successfully.


In [8]:
#@title Upload Reference Audio
from google.colab import files
from IPython.display import clear_output

def upload_audio():
  upload_folder="/content/uploaded_audio"
  os.makedirs(upload_folder,exist_ok=True)
  os.chdir(upload_folder)
  f_names=[]
  uploaded = files.upload()
  for fn in uploaded.keys():
        f_names.append(f"{upload_folder}/{fn}")
  os.chdir("/content/chatterbox")
  clear_output()
  return f_names[-1]
uploaded_audio=upload_audio()
play_audio(uploaded_audio, autoplay=False)
uploaded_audio


'/content/uploaded_audio/female_shadowheart4.flac'

In [17]:
#@title Voice Clone
Text = "Now let's make my mum's favourite. So three mars bars into the pan. Then we add the tuna and just stir for a bit, just let the chocolate and fish infuse. A sprinkle of olive oil and some tomato ketchup. Now smell that. Oh boy this is going to be incredible."  # @param {type: "string"}

Reference_Audio = "/content/uploaded_audio/female_shadowheart4.flac"  # @param {type: "string"}

exaggeration = 0.5  # @param {type: "number"}
cfgw = 0.5  # @param {type: "number"}
temperature = 0.8  # @param {type: "number"}
seed_num = 0  # @param {type: "number"}
cloned_voice_path=generate_and_save_all(Text, Reference_Audio, exaggeration, temperature, seed_num, cfgw)
from IPython.display import clear_output
clear_output()
print(f"Cloned Voice Save at {os.path.abspath(cloned_voice_path)}")
drive_folder = "/content/gdrive/MyDrive/colab_download"

if os.path.exists(drive_folder):
    drive_path = shutil.copy(cloned_voice_path, drive_folder)
    print(f"✅ Cloned voice saved to Google Drive at: {drive_path}")

# cloned_voice_path=generate_and_save_all(chatterbox_model, text, "/content/Recording (5).wav", 0.5, 0.8, 0, 0.5)


#because colab can't handle large audio to play
if len(Text)<=2000:
  play_audio(cloned_voice_path, autoplay=True)
else:
  print("📥 The audio is too long to play automatically. Please download it to listen.")


Cloned Voice Save at /content/chatterbox/cloned_voices/now_lets_make_my_mum_en_D854F25F.wav


In [11]:
#@title Download Audio

from google.colab import files
files.download(cloned_voice_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
#@title Compare Original and Cloned Voice [Don't play large audio on colab]

print("Original Voice:")
play_audio(Reference_Audio)
print("Cloned Voice:")
play_audio(cloned_voice_path)


Original Voice:


Cloned Voice:




### 🔧 **Chatterbox TTS Parameters Made Simple**

| **Parameter**                    | **What It Does**                                                                                                       | **Easy Tips & Examples**                                                                                                 |
| -------------------------------- | ---------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------ |
| **🎭 Exaggeration** (0.25–2.0)   | Controls how expressive or emotional the voice sounds. <br>Low = flat and robotic, High = dramatic and lively.         | 🟢 0.25 = boring, robotic voice<br>🟠 0.5 = normal, natural voice<br>🔴 1.5+ = excited, emotional, like acting in a play |
| **🧠 CFG / Pace** (0.0–1.0)      | Controls how closely the voice matches the reference audio style. <br>Low = more creative freedom, High = strict copy. | 🟢 0.0 = AI adds its own style<br>🔴 1.0 = copies reference voice exactly                                                |
| **🎲 Random Seed** (0 or number) | Makes output repeatable. <br>Same seed + same text = same voice every time. <br>Seed=0 means random every time.        | 🟢 0 = surprise me, different result each time<br>🔢 123 = always get the same voice for the same text                   |
| **🔥 Temperature** (0.05–5.0)    | Controls how creative or “wild” the voice sounds. <br>Low = safe and predictable, High = more varied but can be weird. | 🟢 0.1 = calm, serious voice<br>🟠 0.8 = natural, clear voice<br>🔴 2.0+ = dramatic but might sound odd                  |

---

**Summary:**

* Use **Exaggeration** to add emotion or keep it flat.
* Use **CFG/Pace** to decide if the voice sticks close to the reference or improvises.
* Use **Random Seed** to get repeatable voices or random ones.
* Use **Temperature** to control how predictable or creative the voice sounds.


